In [1]:
# 示例
from PIL import Image
import pickle
import numpy as np
import os
import cv2



In [2]:
def _plot_mask_(image, mask):
    # 创建颜色映射字典，将每个类别映射到不同的颜色
    color_mapping1 = {
        1: (0, 0, 255),  # 蓝色
        2: (0, 255, 0),  # 绿色
        3: (255, 0, 0),  # 红色
        4: (255, 255, 0)  # 黄色
    }
    color_mapping2 = {
        1: hex_to_rgb('#E4C0BE'),  # 浅粉色
        2: hex_to_rgb('#315098'),  # 深蓝色
        3: hex_to_rgb('#FBD0E0'),  # 浅粉红色
        4: hex_to_rgb('#AFBDB0')  # 灰色
    }
    color_mapping3 = {
        1: hex_to_rgb('#E4C0BE'),  # 浅粉色
        2: hex_to_rgb('#315098'),  # 深蓝色
        3: hex_to_rgb('#F8B042'),  # 橙黄色
        4: hex_to_rgb('#AFBDB0')  # 灰色
    }
    color_mapping4 = {
        1: hex_to_rgb('#E4C0BE'),  # 浅粉色
        2: hex_to_rgb('#6495ED'),  # 浅蓝色
        3: hex_to_rgb('#F8B042'),  # 橙黄色
        4: hex_to_rgb('#FF00FF')  # 灰色
    }
    color_mapping5 = {
        1: hex_to_rgb('#E4C0BE'),  # 浅粉色
        2: hex_to_rgb('#93AEC1'),  # 蓝灰色
        3: hex_to_rgb('#F8B042'),  # 橙黄色
        4: hex_to_rgb('#AFBDB0')  # 灰色
    }
    color_mapping6 = {
        1: hex_to_rgb('#E4C0BE'),  # 浅粉色
        2: hex_to_rgb('#84C9EF'),  # 天蓝色
        3: hex_to_rgb('#F8B042'),  # 橙黄色
        4: hex_to_rgb('#AFBDB0')  # 灰色
    }

    color_mapping = color_mapping4
    for class_id in range(1,5):
        class_mask = mask==class_id
        # 获取类别对应的颜色
        color = color_mapping.get(class_id, (0, 0, 0))  # 默认为黑色
        # 使用颜色叠加到原图像上，以半透明的方式
        #image[class_mask] = 0.0 * image[class_mask] + 1.0 * np.array(color, dtype=np.uint8)
        image[class_mask] =  np.array(color, dtype=np.uint8)
    return image

def hex_to_rgb(hex_color):
    # 移除颜色代码中的 # 符号
    hex_color = hex_color.lstrip('#')
    
    # 解析红、绿、蓝分量
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    
    return (r, g, b)


#### 预测的mask

In [63]:
## 画预测mask
import os
# 调色板
palette = [[128, 128, 128], [129, 127, 38], [120, 69, 125], [53, 125, 34], [0, 11, 123]]

model_list = ['unet','upper_r50','twins','swin_upernet','setr','segnext-B','deeplabv3','hrnet','mask2former']

for model in model_list:

    # 指定要遍历的文件夹路径
    folder_path = f'../work_dirs/{model}/pred'
    save_root = f'../work_dirs/{model}/pred_vis'
    img_root = '/media/HDD/hln/dataset/HipJoint_mmseg/images/'
    try:
        os.makedirs(save_root)
    except:
        pass
    # 遍历文件夹
    for filename in os.listdir(folder_path):
        # 打开预测的mask
        file_path = os.path.join(folder_path, filename)
        mask = Image.open(file_path).convert('P')
        mask = np.array(mask)
        # 读取原图
        img_path = filename[:-3]+'jpg'
        img_path = img_root+img_path
        img = Image.open(img_path).convert('RGB')
        img = np.array(img)
        # 上色
        img_mask = _plot_mask_(image=img.copy(),mask=mask)
        img_mask = Image.fromarray(img_mask)
        #image.putpalette(np.array(palette, dtype=np.uint8))
        # 保存图像副本
        save_name = filename
        save_dir = os.path.join(save_root, save_name)
        img_mask.save(save_dir)


#### 预测的热力图

In [6]:
## 画预测mask
import os
# 调色板
palette = [[128, 128, 128], [129, 127, 38], [120, 69, 125], [53, 125, 34], [0, 11, 123]]

model_list = ['unet','upper_r50','twins','swin_upernet','setr','segnext-B','deeplabv3','hrnet','mask2former']
model_list = ['mask2former']
for model in model_list:

    # 指定要遍历的文件夹路径
    folder_path = f'../work_dirs/{model}/pred_logit'
    save_root = f'../work_dirs/{model}/heat_vis/'
    img_root = '/media/HDD/hln/dataset/HipJoint_mmseg/images/'
    try:
        os.makedirs(save_root)
    except:
        pass
    # 遍历文件夹
    for filename in os.listdir(folder_path):
        # 打开预测的heatmap
        file_path = os.path.join(folder_path, filename)
        heatmap = np.load(file_path)['arr_0']
        if model=='mask2former':
            pass
        else:
            heatmap = np.exp(heatmap)
            heatmap = heatmap/heatmap.sum(0,keepdims=True)
        heatmap = heatmap[1:].sum(axis=0)
        heatmap = heatmap*255
        # 读取原图
        img_path = filename[:-3]+'jpg'
        img_path = img_root+img_path
        img = Image.open(img_path).convert('RGB')
        img = np.array(img)
        # 上色
        heatmap = cv2.applyColorMap(heatmap.astype(np.uint8), cv2.COLORMAP_JET)
        #heatmap[:,:,0]=0
        result = img.copy()
        #index = heatmap>10
        result = result*0.4 + heatmap*0.6

        cv2.imwrite(save_root+filename[:-3]+'png', result)



#### label mask

In [3]:
## 画label mask

## 画预测mask
import os
# 调色板
palette = [[128, 128, 128], [129, 127, 38], [120, 69, 125], [53, 125, 34], [0, 11, 123]]


# 指定要遍历的文件夹路径
folder_path = f'/media/HDD/hln/dataset/HipJoint_mmseg/labels/'
save_root = f'../work_dirs/label/pred_vis'
img_root = '/media/HDD/hln/dataset/HipJoint_mmseg/images/'
try:
    os.makedirs(save_root)
except:
    pass
# 遍历文件夹
for filename in os.listdir(folder_path):
    # 打开预测的mask
    file_path = os.path.join(folder_path, filename)
    mask = Image.open(file_path).convert('P')
    mask = np.array(mask)
    # 读取原图
    img_path = filename[:-3]+'jpg'
    img_path = img_root+img_path
    img = Image.open(img_path).convert('RGB')
    img = np.array(img)
    # 上色
    img_mask = _plot_mask_(image=img.copy(),mask=mask)
    img_mask = Image.fromarray(img_mask)
    #image.putpalette(np.array(palette, dtype=np.uint8))
    # 保存图像副本
    save_name = filename
    save_dir = os.path.join(save_root, save_name)
    img_mask.save(save_dir)


#### label heatmap

In [66]:
## 画label mask

## 画预测mask
import os
# 调色板
palette = [[128, 128, 128], [129, 127, 38], [120, 69, 125], [53, 125, 34], [0, 11, 123]]


# 指定要遍历的文件夹路径
folder_path = f'/media/HDD/hln/dataset/HipJoint_mmseg/labels/'
save_root = f'../work_dirs/label/heat_vis/'
img_root = '/media/HDD/hln/dataset/HipJoint_mmseg/images/'
try:
    os.makedirs(save_root)
except:
    pass
# 遍历文件夹
for filename in os.listdir(folder_path):
    # 打开预测的mask
    file_path = os.path.join(folder_path, filename)
    mask = Image.open(file_path).convert('P')
    mask = np.array(mask)
    mask[mask>=1] = 255
    # 读取原图
    img_path = filename[:-3]+'jpg'
    img_path = img_root+img_path
    img = Image.open(img_path).convert('RGB')
    img = np.array(img)
    # 上色
    heatmap = cv2.applyColorMap(mask.astype(np.uint8), cv2.COLORMAP_JET)
    result = img.copy()
    #index = heatmap>10
    result = result*0.4 + heatmap*0.6
    cv2.imwrite(save_root+filename[:-3]+'png', result)


#### 迁移文件夹

In [5]:
import os
import shutil

# 定义目录A和目录B的路径
dir_source = '/home/huliangni/Segmentation/mmsegmentation/HipJoint/work_dirs'
dir_des = '/media/HDD/hln/TMI/seg_visual/'

#model_list =  ['unet','upper_r50','twins','swin_upernet','setr','segnext-B','deeplabv3','hrnet','mask2former','label']
model_list = ['label']
# 定义要复制的子文件夹列表
subfolders = ['heat_vis', 'pred_vis']

for model in model_list:

    # 在目录B中创建hrnet文件夹的路径
    dir_des_model = os.path.join(dir_des, model)

    # 如果hrnet文件夹不存在，则创建它
    if not os.path.exists(dir_des_model):
        os.makedirs(dir_des_model)

    # 遍历子文件夹列表并复制每个文件夹
    for subfolder in subfolders:
        # 源文件夹路径
        source = os.path.join(dir_source, model, subfolder)
        # 目标文件夹路径
        destination = os.path.join(dir_des_model, subfolder)
        
        # 如果源文件夹存在，则复制到目标路径
        if os.path.exists(source):
            shutil.copytree(source, destination)
        else:
            print(f"Source folder {source} does not exist, skipping.")

    print(f"Copy {model} complete.")

Copy label complete.
